<a href="https://colab.research.google.com/github/zhiruo159/ISOM3260_DatabaseProject/blob/main/%5BISOM3260%5D_Member_Manager_Finalcode_firstDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -- Install and setup Oracle Instant Client --
! wget -O oracleinstantclient.zip https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
! unzip -oq oracleinstantclient.zip
! mkdir -p /opt/oracle
! mv instantclient_19_6 /opt/oracle/
! sh -c "echo /opt/oracle/instantclient_19_6 > /etc/ld.so.conf.d/oracle-instantclient.conf"
! ldconfig


# -- Include oracledb package --
! pip install oracledb

--2025-11-07 04:53:47--  https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
Resolving download.oracle.com (download.oracle.com)... 23.219.72.96
Connecting to download.oracle.com (download.oracle.com)|23.219.72.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75462547 (72M) [application/zip]
Saving to: ‘oracleinstantclient.zip’

oracleinstantclient 100%[===================>]  71.97M  60.1MB/s    in 1.2s    

2025-11-07 04:53:49 (60.1 MB/s) - ‘oracleinstantclient.zip’ saved [75462547/75462547]

mv: cannot move 'instantclient_19_6' to '/opt/oracle/instantclient_19_6': Directory not empty
/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_loader.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_opencl.so.0 is not a symbolic l

In [ ]:
# Import oracledb package and connect to Oracle Database
import oracledb

HOST_NAME = "imz409.ust.hk"
PORT_NUMBER = "1521"
SERVICE_NAME = "imz409"
USERNAME = "kcloaf"
PASSWORD = "7745"

dsn_tns = oracledb.makedsn(HOST_NAME, PORT_NUMBER, service_name=SERVICE_NAME)
conn = oracledb.connect(user=USERNAME, password=PASSWORD, dsn=dsn_tns)

c = conn.cursor()

In [ ]:
from google.colab.widgets import TabBar
import ipywidgets as widgets
import time
import pandas as pd
from ipywidgets import interact
from ipywidgets import Text
from ipywidgets import DatePicker
from ipywidgets import IntText
from ipywidgets import Label
from datetime import date, datetime

# for Manager Dashboard
#from ipywidgets import Output
from IPython.display import display
from sqlalchemy import create_engine
#import time


# Set current_user_id = None to tell every part of the program "no one is logged in yet"
# It prevents **TyperError: can only concatenate str (not "NoneType") to str** that might happen later on at load_kid_information() tied to build the SQL string while current_user_id was still undefined (or None)
current_user_id = None


# Issac: "I haven't add the tabs for manager at this moment yet"
tb_all = TabBar(['Home Page', 'Manager Login Page', 'Member Login Page', 'Member Register Page',
                 'Main Menu', 'Personal Info', 'Forget Password Page',
                 'Reset Password Page', 'My Kid Information', 'Manager Dashboard', 'Add Teacher',
                 'Edit Teacher', 'Delete Teacher', 'Add Course', 'Edit Course', 'Delete Course'])

# a simple function for jumping between tabs
def showPage(tabName):
    with tb_all.output_to(tabName):
        display()


#######################################################
################ Home Page Tab - Begin ################
#######################################################
with tb_all.output_to('Home Page'):

    # create 3 labels
    title_label = Label("Happy Creche Center Management System")
    subtitle_label = Label("Welcome to our Happy Creche Center Management portal")
    role_label = Label("Please select your role")

    # create 2 button
    btnManagerLoginPage = widgets.Button(description="Continue as Manager")
    btnMemberLoginPage = widgets.Button(description="Continue as Parent")

    # define the event handlers for the two buttons when they are being clicked
    def on_btnManagerLoginPage_button_clicked(b):
        showPage("Manager Login Page")
    btnManagerLoginPage.on_click(on_btnManagerLoginPage_button_clicked)

    def on_btnMemberLoginPage_button_clicked(b):
        showPage("Member Login Page")
    btnMemberLoginPage.on_click(on_btnMemberLoginPage_button_clicked)

    # display the objects
    homePage_row1HBox = widgets.HBox([title_label])
    homePage_row2HBox = widgets.HBox([subtitle_label])
    homePage_row3HBox = widgets.HBox([role_label])
    homePage_row4HBox = widgets.HBox([btnManagerLoginPage])
    homePage_row5HBox = widgets.HBox([btnMemberLoginPage])

    display(widgets.VBox([homePage_row1HBox, homePage_row2HBox, homePage_row3HBox, homePage_row4HBox, homePage_row5HBox]))
################ Home Page Tab - End ################

#######################################################
################ Main Menu Tab - Begin ################
#######################################################
with tb_all.output_to('Main Menu'):

    # create 1 label
    mainMenu_label = Label("Main Menu")
    spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")

    # create 3 button
    btnKid_InfoPage = widgets.Button(description="Edit my kid Information")
    btnMyProfilePage = widgets.Button(description="Edit my Profile")
    btnloginPage = widgets.Button(description="Logout")

    def on_btnKid_InfoPage_button_clicked(b):
        showPage("My Kid Information")
        load_kid_information()  # immediately load kid data
    btnKid_InfoPage.on_click(on_btnKid_InfoPage_button_clicked)

    def on_btnMyProfilePage_button_clicked(b):
        showPage("Personal Info")
        load_member_profile_information()  # immediately load member data
    btnMyProfilePage.on_click(on_btnMyProfilePage_button_clicked)

    def on_btnloginPage_button_clicked(b):
      showPage("Member Login Page")
    btnloginPage.on_click(on_btnloginPage_button_clicked)

    # display the objects
    mainMenu1VBox = widgets.VBox([mainMenu_label])
    mainMenu1HBox = widgets.HBox([btnKid_InfoPage, btnMyProfilePage])
    mainMenu2VBox = widgets.VBox([spacer])
    mainMenu2HBox = widgets.HBox([btnloginPage])

    display(widgets.VBox([mainMenu1VBox, mainMenu1HBox, mainMenu2VBox, mainMenu2HBox]))
################ Main Menu Tab - End ################


###########################################################
########### Member Register Page Tab - Begin ##############
###########################################################
with tb_all.output_to('Member Register Page'):

  # create labels and input widgets
  INS_FirstName_label = Label("First Name: ")
  INS_FirstName_textbox = Text(placeholder='First Name', disabled=False)  # False --> textbox is enable, users can type into it

  INS_LastName_label = Label("Last Name: ")
  INS_LastName_textbox = Text(placeholder='Last Name', disabled=False)

  INS_HomeAddress_label = Label("Home Address: ")
  INS_HomeAddress_textbox = Text(placeholder='Home Address', disabled=False)

  INS_DOB_label = Label("DOB: ")
  INS_DOB_datepicker = DatePicker(placeholder='DOB', disabled=False)

  INS_telephone_label = Label("Telephone: ")
  INS_telephone_textbox = Text(placeholder='Telephone', disabled=False)

  INS_email_label = Label("Email: ")
  INS_email_textbox = Text(placeholder='Email', disabled=False)

  INS_password_label = Label("Password: ")
  INS_password_textbox = Text(placeholder='Password', disabled=False)

  Reg_spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")

  INS_Kid_F_Name_label = Label("Kid First Name: ")
  INS_Kid_F_Name_textbox = Text(placeholder='Kid First Name', disabled=False)

  INS_Kid_L_Name_label = Label("Kid Last Name: ")
  INS_Kid_L_Name_textbox = Text(placeholder='Kid Last Name', disabled=False)

  INS_Kid_DOB_label = Label("Kid DOB: ")
  INS_Kid_DOB_datepicker = DatePicker(placeholder='Kid DOB', disabled=False)


  INS_secQ_label = Label("Security Questions")
  INS_secQ_list = widgets.Dropdown(
      options=[
          "What is your favorite fruit?",
          "What is the name of your first pet?",
          "Which country did you travel for the first time?"
      ]
  )

  INS_secAns_label = Label("Security Question Answer")
  INS_secAns_textbox = Text(placeholder='Security Question Answer', disabled=False)

  # create 3 buttons
  INS_back_button = widgets.Button(description="Back")
  INS_clear_button = widgets.Button(description="Clear")
  INS_register_button = widgets.Button(description="Register")

  # event handlers
  def on_INS_back_button_clicked(b):
      showPage("Home Page")
  INS_back_button.on_click(on_INS_back_button_clicked)

  def on_INS_clear_button_clicked(b):
      INS_FirstName_textbox.value = ""
      INS_LastName_textbox.value = ""
      INS_HomeAddress_textbox.value = ""
      INS_DOB_datepicker.value = None
      INS_telephone_textbox.value = ""
      INS_email_textbox.value = ""
      INS_password_textbox.value = ""
      INS_Kid_F_Name_textbox.value = ""
      INS_Kid_L_Name_textbox.value = ""
      INS_Kid_DOB_datepicker.value = ""
      INS_secAns_textbox.value = ""
      INS_secQ_list.value = INS_secQ_list.options[0]
      INS_register_button.disabled = False
  INS_clear_button.on_click(on_INS_clear_button_clicked)

  def on_INS_register_button_clicked(b):
      # generate next Mxxx ID
      c.execute("SELECT MEM_ID FROM MEMBER ORDER BY MEM_ID DESC") # Get all member ID, sort from highest to lowest "DESC" --> M002, M001...
      row = c.fetchone()  # Return only the first row from the query (tuple) result (i.e. all IDs are arranged vertically from the list)
      if row is None:
          MEM_ID = "M001"
      else:
          last_id = row[0]            #M005
          num = int(last_id[1:]) + 1  # --> int('005') + 1 --> 5+1 --> =6
          MEM_ID = f"M{num:03d}"      # 0 --> pad with zeros instead of empty spaces
                                      # 3 --> minimum width of 3 characters
                                      # d --> decimal integer (whole number)

    # insert into database - 14 values, including SYSDATE for MEM_REG_DATE
      c.execute(
        """INSERT INTO MEMBER VALUES ('{}', '{}', '{}', '{}', TO_DATE('{}', 'YYYY-MM-DD'),
                                     '{}', '{}', TO_DATE('{}', 'YYYY-MM-DD'),
                                     '{}', '{}', '{}', '{}', '{}', SYSDATE)"""
        .format(
            MEM_ID,                                           # 1. MEM_ID
            INS_password_textbox.value.strip(),               # 2. MEM_PW
            INS_FirstName_textbox.value.strip(),              # 3. MEM_FNAME
            INS_LastName_textbox.value.strip(),               # 4. MEM_LNAME

            # strftime --> str f time --> string from time = "Create a string from time/date object"
            # If a date is selected, convert it to 'YYYY-MM-DD' string; If not, use empty string (Null in database)
            INS_DOB_datepicker.value.strftime('%Y-%m-%d') if INS_DOB_datepicker.value else '',  # 5. MEM_DOB
            INS_Kid_F_Name_textbox.value.strip(),             # 6. K_FNAME
            INS_Kid_L_Name_textbox.value.strip(),             # 7. K_LNAME
            INS_Kid_DOB_datepicker.value.strftime('%Y-%m-%d') if INS_Kid_DOB_datepicker.value else '',  # 8. K_DOB
            INS_secQ_list.value,                              # 9. MEM_SECURITY_Q
            INS_secAns_textbox.value.strip(),                 # 10. MEM_SECURITY_ANS
            INS_telephone_textbox.value.strip(),              # 11. MEM_TEL
            INS_HomeAddress_textbox.value.strip(),            # 12. MEM_ADD
            INS_email_textbox.value.strip()                   # 13. MEM_EMAIL
            # 14. MEM_REG_DATE → SYSDATE (auto-filled)
         )
      )
      c.execute("commit")
      INS_register_button.disabled = True

      # hide form
      INS_line1HBox.layout.display = 'none'
      Reg_spacer.layout.display = 'none'
      INS_line2HBox.layout.display = 'none'
      INS_line2HBox.layout.display = 'none'

      # SUCCESS MESSAGE + CONTINUE BUTTON
      msg = Label("Done!\nYour account has been set!\nYour member ID is: " + MEM_ID)
      INS_continue_button = widgets.Button(description="Continue")

      # Group them in a container
      success_container = widgets.VBox([msg, INS_continue_button])
      display(success_container)

      # Define continue action with cleanup
      def on_continue_clicked(b):
          success_container.close()   # Remove message & button
          showPage("Home Page")

      INS_continue_button.on_click(on_continue_clicked)

  INS_register_button.on_click(on_INS_register_button_clicked)

  # layout

  INS_P_labelVBox = widgets.VBox([
      INS_FirstName_label, INS_LastName_label, INS_HomeAddress_label,
      INS_DOB_label, INS_telephone_label, INS_email_label,
      INS_password_label, INS_secQ_label, INS_secAns_label
  ])

  INS_K_labelVBox = widgets.VBox([
      INS_Kid_F_Name_label, INS_Kid_L_Name_label, INS_Kid_DOB_label
  ])

  INS_P_textboxVBox = widgets.VBox([
      INS_FirstName_textbox, INS_LastName_textbox, INS_HomeAddress_textbox,
      INS_DOB_datepicker, INS_telephone_textbox, INS_email_textbox,
      INS_password_textbox, INS_secQ_list, INS_secAns_textbox
  ])

  INS_K_textboxVBox = widgets.VBox([
      INS_Kid_F_Name_textbox, INS_Kid_L_Name_textbox, INS_Kid_DOB_datepicker
  ])

  INS_line1HBox = widgets.HBox([INS_P_labelVBox, INS_P_textboxVBox])
  INS_line2HBox = widgets.HBox([INS_K_labelVBox, INS_K_textboxVBox])
  INS_line3HBox = widgets.HBox([INS_back_button, INS_clear_button, INS_register_button])


  display(widgets.VBox([INS_line1HBox,
                        Reg_spacer,
                        INS_line2HBox,
                        INS_line3HBox
                        ]))

############# Member Register Page Tab - End #############


###########################################################
############ Member Login Page Tab - Begin ###############
###########################################################
with tb_all.output_to('Member Login Page'):

    # create 2 textboxes and 2 label
    login_error_label = Label(value='')
    login_label = Label("Login")
    id_textbox = Text(value='', placeholder='Enter a Member ID', description='Member ID:', disabled=False)
    pw_textbox = Text(description='Password:', placeholder='Enter your Password', disabled=False)

    # create 4 buttons
    login_button = widgets.Button(description="Login")
    cancel_button = widgets.Button(description="Cancel")
    forget_pw_button = widgets.Button(description="Forget Password")
    notyetregister_button = widgets.Button(description="Not yet Register?")

    def on_login_button_clicked(b):
        # clear previous error
        error_label = widgets.Label(value="")
        error_label.value = ""

        # make global variable accessible
        global current_user_id

        mem_id = id_textbox.value.strip()
        mem_pw = pw_textbox.value.strip()

        # 1. Check empty fields
        if not mem_id or not mem_pw:
            login_error_label.value = "Please fill in both fields."
            return

        # 2. Check if Member ID exists
        c.execute("SELECT * FROM MEMBER WHERE MEM_ID = '" + mem_id + "'")
        result = c.fetchone()

        if result is None:
          login_error_label.value = "Member not registered. Please register first"
          return

        db_pw = result[1]

        # 3. Check password
        if db_pw == mem_pw:
          current_user_id = mem_id
          pw_textbox.value = ""
          login_error_label.value = ""
          showPage("Main Menu")
        else:
          login_error_label.value = "Invalid Member ID or Password"

    # connect the button to this function
    login_button.on_click(on_login_button_clicked)

    def on_cancel_button_clicked(b):
        showPage("Home Page")
    cancel_button.on_click(on_cancel_button_clicked)

    def on_forget_pw_button_clicked(b):
        showPage("Forget Password Page")
    forget_pw_button.on_click(on_forget_pw_button_clicked)

    def on_notyetregister_button_clicked(b):
        showPage("Member Register Page")
    notyetregister_button.on_click(on_notyetregister_button_clicked)

    # display the objects
    member_login1HBox = widgets.HBox([login_label])
    member_login1VBox = widgets.VBox([id_textbox, pw_textbox])
    member_login2HBox = widgets.HBox([login_button, cancel_button, forget_pw_button, notyetregister_button])

    display(widgets.VBox([member_login1HBox, member_login1VBox, member_login2HBox, login_error_label]))
############# Member Login Page Tab - End ##############


##########################################################
#################### Forget password #####################
##########################################################
with tb_all.output_to('Forget Password Page'):

    # Labels + inputs
    forget_pw_label = Label("Forget your password?")
    fpw_label = Label("Registered email:     ")
    fpw_textbox = Text(value='', placeholder='Enter your registered email', disabled=False)
    fpw_enter_button = widgets.Button(description="Enter")

    spacer = Label("────────────────────────────────────────")

    security_Q_label = Label("Security question:     ")
    security_Q_textbox = Text(disabled=True) # True --> textbox is disabled, users cannot type into it

    security_ans_label = Label("Answer:     ")
    security_ans_textbox = Text(value='', placeholder='Enter your answer', disabled=True)

    security_ans_enter_button = widgets.Button(description="Enter", disabled=True)
    fpw_back_button = widgets.Button(description="Back")

    error_label = widgets.Label(value="")

    # Store member data globally for answer check
    member_row = None  # Will hold the full row when email is found

    # Show security question + enable answer field
    def on_fpw_enter_button_clicked(b):
        global member_row
        email = fpw_textbox.value.strip()

        if not email:
            error_label.value = "Please fill in your registered email."
            return

        # Clear previous
        error_label.value = ""
        security_Q_textbox.value = ""
        security_ans_textbox.value = ""
        security_ans_textbox.disabled = True
        security_ans_enter_button.disabled = True
        member_row = None

        # Query member by email
        c.execute("SELECT * FROM MEMBER WHERE MEM_EMAIL = '" + email + "'")
        row = c.fetchone()

        if row:   # If a row was returned from the database (i.e., the email exists)
            # Store full row for later answer check
            member_row = row
            # Show security question (column 9 = MEM_SECURITY_Q)
            security_Q_textbox.value = row[8]  # 0-indexed: 9th column
            # Enable answer field
            security_ans_textbox.disabled = False
            security_ans_enter_button.disabled = False
        else:
            error_label.value = "Invalid Email."

    fpw_enter_button.on_click(on_fpw_enter_button_clicked)

    # Check answer using stored row
    def on_security_ans_enter_button_clicked(b):
        global member_row
        answer = security_ans_textbox.value.strip()

        if not answer:
            error_label.value = "Please fill in your answer."
            return

        if member_row and answer == member_row[9]:  # 10th column = MEM_SECURITY_ANS
            showPage("Reset Password Page")
        else:
            error_label.value = "Wrong answer, please enter again."

    security_ans_enter_button.on_click(on_security_ans_enter_button_clicked)

    def on_fpw_back_button_clicked(b):
        showPage("Member Login Page")
    fpw_back_button.on_click(on_fpw_back_button_clicked)

    # Layout
    security_labelVBox = widgets.VBox([security_Q_label, security_ans_label])
    security_textboxVBox = widgets.VBox([security_Q_textbox, security_ans_textbox])
    fpw_line1HBox = widgets.HBox([fpw_label, fpw_textbox])
    fpw_line2HBox = widgets.HBox([security_labelVBox, security_textboxVBox])
    fpw_line3HBox = widgets.HBox([fpw_enter_button, fpw_back_button])

    # Display
    display( widgets.VBox([ fpw_line1HBox, fpw_line3HBox,
        ]),
        error_label,
        spacer,
        widgets.HBox([fpw_line2HBox]),
        security_ans_enter_button
    )

############# Forget Password Page Tab - End ##############

###########################################################
################## Update Personal Info ###################
###########################################################
with tb_all.output_to('Personal Info'):  # when I click edit my profile I was directed to Personal Info tab from main menu

    # Title
    View_info_page_label = Label("View/Edit My Profile")

    # Load existing information for the given Member ID as member already login
    def load_member_profile_information():
        global current_user_id
        c.execute(
            "SELECT MEM_ADD, TO_CHAR(MEM_DOB,'YYYY-MM-DD'), MEM_TEL, MEM_EMAIL, MEM_PW "
            "FROM MEMBER WHERE MEM_ID = '" + current_user_id + "'"
        )

        for row in c:   # Loop through each row returned by the database query
            txtAddress.value = row[0]
            DOB_datepicker.value = datetime.strptime(row[1], "%Y-%m-%d").date() if row[1] else None
            txtTel.value = row[2]
            txtEmail.value = row[3]
            txtPW.value = row[4]

    # Buttons
    back_main_button = widgets.Button(description="Back to Main Menu")
    Edit_member_profile_button = widgets.Button(description="Confirm Edit")

    # Prompt for user input of new updated profile information
    lblAddr = Label("Home Address:")
    txtAddress = widgets.Text(placeholder="Enter new address")

    lblDOB = Label("DOB (YYYY-MM-DD):")
    DOB_datepicker = DatePicker(placeholder="DOB (YYYY-MM-DD):", disabled=False)  # disabled = false will enable the datepicker

    lblTel = Label("Telephone:")
    txtTel = widgets.Text(placeholder="Enter new telephone")

    lblEmail = Label("Email:")
    txtEmail = widgets.Text(placeholder="Enter new email")

    lblPW = Label("Password:")
    txtPW = widgets.Text(placeholder="Enter new password")


    def on_back_main_button_clicked(b):
        showPage("Main Menu")

    # message label used to display successful edit
    confirm_output_label = Label(value='')

    def on_Edit_member_profile_button_clicked(b):
        c.execute(
            """
            UPDATE MEMBER
            SET MEM_ADD = '{}',
                MEM_DOB = TO_DATE('{}', 'YYYY-MM-DD'),
                MEM_TEL = '{}',
                MEM_EMAIL = '{}',
                MEM_PW = '{}'
            WHERE MEM_ID = '{}'
            """.format(
                txtAddress.value.strip(),
                DOB_datepicker.value.strftime('%Y-%m-%d'),
                txtTel.value.strip(),
                txtEmail.value.strip(),
                txtPW.value.strip(),
                current_user_id
            )
        )
        c.execute("commit")
        confirm_output_label.value = "Your profile has been successfully updated."

    back_main_button.on_click(on_back_main_button_clicked)
    Edit_member_profile_button.on_click(on_Edit_member_profile_button_clicked)

    # Layout
    # Arrange labels and inputs in two vertical boxes side by side
    member_labelVBox = widgets.VBox([lblAddr, lblDOB, lblTel, lblEmail, lblPW])
    member_inputVBox = widgets.VBox([txtAddress, DOB_datepicker, txtTel, txtEmail, txtPW])

    # Combine the label and input columns horizontally
    member_line1HBox = widgets.HBox([member_labelVBox, member_inputVBox])

    # Buttons line
    member_line2HBox = widgets.HBox([back_main_button, Edit_member_profile_button])

    # Display layout
    display(View_info_page_label, member_line1HBox, member_line2HBox, confirm_output_label)
############# Update Personal Info Tab - End ##############

##########################################################
############ Reset Password Page Tab - Begin ###############
##########################################################
with tb_all.output_to('Reset Password Page'):

  # Labels + inputs
  reset_pw_label = Label("Reset your password")

  rp_new_pw_label = Label("New Password:     ")
  rp_new_pw_textbox = Text(value='', placeholder='Enter new password', disabled=False)

  rp_re_pw_label = Label("Re-enter New Password:     ")
  rp_re_pw_textbox = Text(value='', placeholder='Re-enter new password', disabled=False)

  # create buttons
  rp_back_button = widgets.Button(description="Back")
  rp_confirm_button = widgets.Button(description="Confirm")

  error_label = widgets.Label(value="")

  # Actions
  def on_rp_confirm_button_clicked(b):
    error_label.value = ""

    new_pw = rp_new_pw_textbox.value.strip()
    re_pw  = rp_re_pw_textbox.value.strip()

    if not new_pw or not re_pw:
        error_label.value = "Please fill in both password fields."
        return

    if new_pw != re_pw:
        error_label.value = "Passwords do not match."
        return

    # Retrieve email from Forget Password Page (still in scope)
    email = fpw_textbox.value.strip()

    if not email:
        error_label.value = "Email not found. Please go back and re-enter."
        return

    # Fetch current password (do NOT display it)
    c.execute("SELECT MEM_PW FROM MEMBER WHERE mem_email = '" + email + "'")
    row = c.fetchone()

    if not row:
        error_label.value = "Invalid email or account not found."
        return

    current_pw = row[0]

    if new_pw == current_pw:
        error_label.value = "New password must be different from current password."
        return

    # Update password
    c.execute("UPDATE MEMBER SET MEM_PW = '" + new_pw + "' WHERE mem_email = '" + email + "'")
    c.execute("commit")

    error_label.value = "Password updated successfully!"
    rp_confirm_button.disabled = True
    rp_new_pw_textbox.disabled = True
    rp_re_pw_textbox.disabled = True
  rp_confirm_button.on_click(on_rp_confirm_button_clicked)


  def on_rp_back_button_clicked(b):
      showPage("Forget Password Page")
  rp_back_button.on_click(on_rp_back_button_clicked)

  # Layout
  rp_labelVBox = widgets.VBox([rp_new_pw_label,rp_re_pw_label])
  rp_textboxVBox = widgets.VBox([rp_new_pw_textbox,rp_re_pw_textbox])
  rp_line1HBox = widgets.HBox([rp_labelVBox, rp_textboxVBox])
  rp_line2HBox = widgets.HBox([rp_back_button, rp_confirm_button])

  display(widgets.VBox([
      rp_line1HBox,
      rp_line2HBox,
      error_label
  ]))
############# Reset Password Page Tab - End ##############


##########################################################
############## Edit My Kid Information ###################
##########################################################

with tb_all.output_to('My Kid Information'):

    kid_label = Label("View/Edit My Kid Information")

    # Load existing information for the given Member ID's kid as member already login
    def load_kid_information():
        global current_user_id

        c.execute("SELECT K_FNAME, K_LNAME, TO_CHAR(K_DOB,'YYYY-MM-DD') "
                  "FROM MEMBER WHERE MEM_ID = '" + current_user_id + "'")

        for row in c:
            txtK_FNAME.value = row[0]
            txtK_LNAME.value = row[1]
            DOB_kid_datepicker.value = datetime.strptime(row[2], "%Y-%m-%d").date() if row[2] else None

    # Buttons
    back_main_button = widgets.Button(description="Back to Main Menu")
    Edit_kid_info_button = widgets.Button(description="Confirm Edit")
    lblMsg = Label("")

    # Prompt for user input of new updated kid information
    lblK_FNAME = Label("Kid's First Name:")
    txtK_FNAME = widgets.Text(placeholder="Enter first name")

    lblK_LNAME = Label("Kid's Last Name:")
    txtK_LNAME = widgets.Text(placeholder="Enter last name")

    lblK_DOB = Label("Kid's DOB:")
    DOB_kid_datepicker = DatePicker(placeholder="YYYY-MM-DD", disabled=False)

    # Button Event Handlers
    def on_back_main_button_clicked(b):
        showPage("Main Menu")

    def on_btnSaveKidInfo_clicked(b):
        global current_user_id
        c.execute(
            "UPDATE MEMBER SET K_FNAME = '{}', K_LNAME = '{}', K_DOB = TO_DATE('{}','YYYY-MM-DD') "
            "WHERE MEM_ID = '{}'".format(
                txtK_FNAME.value.strip(),
                txtK_LNAME.value.strip(),
                DOB_kid_datepicker.value.strftime('%Y-%m-%d'),
                current_user_id
            )
        )
        c.execute("commit")
        lblMsg.value = " Kid Information Updated Successfully!"

    Edit_kid_info_button.on_click(on_btnSaveKidInfo_clicked)
    back_main_button.on_click(on_back_main_button_clicked)

    # Layout
    kid_labelVBox = widgets.VBox([lblK_FNAME, lblK_LNAME, lblK_DOB])
    kid_inputVBox = widgets.VBox([txtK_FNAME, txtK_LNAME, DOB_kid_datepicker])
    kid_line1HBox = widgets.HBox([kid_labelVBox, kid_inputVBox])
    kid_line2HBox = widgets.HBox([back_main_button, Edit_kid_info_button])

    display(widgets.VBox([
        kid_label,
        kid_line1HBox,
        kid_line2HBox,
        lblMsg
    ]))

########################################################### MANAGER BELOW ###################################################################


##########################################################
################ Manager Login Tab - Begin ################
##########################################################
with tb_all.output_to('Manager Login Page'):
    # Create widgets
    title_label = Label("Manager Login")

    manager_id = Text(placeholder="Enter your assigned ID", description="Manager ID:")
    manager_password = Text(placeholder="Enter your password", description="Password:")
    error_msg = Label("Invalid Manager ID or Password", style={'color': 'red'})
    error_msg.layout.visibility = 'hidden'

    login_btn = widgets.Button(description="Login", button_style='primary')
    back_btn = widgets.Button(description="Back")

    # Event handlers
    def on_login_click(b):
      # clear previous error
      error_msg.layout.visibility = 'hidden'

      man_id = manager_id.value.strip()
      man_pw = manager_password.value.strip()

      if not man_id or not man_pw:
        error_msg.value = "Please fill in both fields."
        error_msg.layout.visibility = 'visible'
        return


      c.execute("SELECT * FROM MANAGER WHERE MAN_ID = '" + man_id + "' AND MAN_PW = '" + man_pw + "'")

      # Check existence with loop
      found = False
      for row in c:
          found = True
          break  # no need for more, since we just check if match exists

      if found:                     # match found
          error_msg.layout.visibility = 'hidden' # clear error on success
          showPage("Manager Dashboard")
      else:
          error_msg.value = "Invalid Manager ID or Password."
          error_msg.layout.visibility = 'visible'


    def on_back_click(b):
        showPage("Home Page")

    login_btn.on_click(on_login_click)
    back_btn.on_click(on_back_click)

    # Layout
    manager_login1HBox = widgets.HBox([title_label])
    manager_login2VBox = widgets.VBox([manager_id, manager_password])
    manager_login2HBox = widgets.HBox([login_btn, back_btn])


    display(widgets.VBox([manager_login1HBox, manager_login2VBox, manager_login2HBox, error_msg]))
################ Manager Login Tab - End ################



###############################################################
################ Manager Dashboard Tab - Begin ################
##############################################################
with tb_all.output_to('Manager Dashboard'):
    # Create widgets
    title_label = Label("Manager Dashboard")

    logout_btn = widgets.Button(description="Logout")

    teachers_label = Label("Teacher Accounts")
    add_teacher_btn = widgets.Button(description="Add New Teacher")
    edit_teacher_btn = widgets.Button(description="Edit Teacher")
    delete_teacher_btn = widgets.Button(description="Delete Teacher")

    courses_label = Label("Course Details")
    add_course_btn = widgets.Button(description="Add New Course")
    edit_course_btn = widgets.Button(description="Edit Course")
    delete_course_btn = widgets.Button(description="Delete Course")
    refresh_btn = widgets.Button(description = 'Refresh the Page')

    # Create tables (simplified as text areas for demo)
    teachers_table = widgets.Textarea(description="Teachers:", disabled=True, layout={'width': '100%', 'height': '200px'})
    courses_table = widgets.Textarea(description="Courses:", disabled=True, layout={'width': '100%', 'height': '300px'})

    def update_tables():
        # Update teachers table
        #teachers_text = "ID\tName\t\t\tEmail\t\t\t\tStatus\n"
        #teachers_text += "-" * 80 + "\n"

        connection_url = f"oracle+oracledb://{USERNAME}:{PASSWORD}@{HOST_NAME}:{PORT_NUMBER}/?service_name={SERVICE_NAME}"
        engine = create_engine(connection_url)

        sql_query = "SELECT * FROM TEACHER"
        df = pd.read_sql(sql_query, engine)
        #print(df.columns.tolist())
        df_selected = df[['ter_id', 'ter_lname', 'ter_fname','ter_email','ter_status']]
        df_selected = df_selected.rename(columns={'ter_id': 'ID', "ter_lname":"Last Name", "ter_fname": "First Name","ter_email" : "Email","ter_status": "Status" })
        #out = Output()

        #for index in range(len(df)):
          #teachers_text += f"{df.iloc[index,0]}\t{df.iloc[index, 1]} {df.iloc[index, 2]}\t{df.iloc[index, 3]}\t{df.iloc[index, 5]}\n"
        #teachers_table.value = teachers_text
        teachers_table.value = df_selected.to_markdown(index = False)
        # Update courses table

        #out = Output()

        #courses_text = "ID\tName\t\t\tDate\t   Time\t\tTeacher\t Age\tQuota\n"
        #courses_text += "-" * 80 + "\n"
        sql_query1 = "SELECT * FROM COURSES"
        df_course = pd.read_sql(sql_query1, engine)
        #df_course_selected = df_course[["cou_id", "cou_name", "cou_date", "teacher_ter_id", "cou_age_min","cou_age_max", "cou_quota"]]

        df_course["cou_age_range"] = (
            df_course["cou_age_min"].astype(str) + "-" +
            df_course["cou_age_max"].astype(str)
        )

        # inplace = False --> Return a NEW DataFrame, don't change the original
        df_course =  df_course.drop(columns=["cou_age_min", "cou_age_max","cou_desc","cou_enrollment_deadline","cou_remarks","cou_category"], inplace=False)

        df_course = df_course.rename(columns={"cou_id":"ID", "cou_name":"Name", "cou_date":"Date", "teacher_ter_id":"Teacher", "cou_age_range" : "Age", "cou_quota":"Quota", "cou_status" : "Status"})

        #for index1 in range(len(df_course)):

          #courses_text += f"{df_course.iloc[index1, 0]}\t{df_course.iloc[index1, 1]}\t{df_course.iloc[index1, 4]}\t" \
          #f"{df_course.iloc[index1, 11]}\t {df_course.iloc[index1, 8]}" \
          #f"-{df_course.iloc[index1, 7]}\t{df_course.iloc[index1, 3]}\n"

        #courses_table.value = courses_text
        courses_table.value = df_course.to_markdown(index = False)

    # Event handlers
    def on_logout_click(b):
        showPage("Home Page")
    logout_btn.on_click(on_logout_click)

    def on_refresh_btn_click(b):
        update_tables()
    refresh_btn.on_click(on_refresh_btn_click)

    def on_add_teacher_click(b):
        showPage("Add Teacher")
    add_teacher_btn.on_click(on_add_teacher_click)

    def on_edit_teacher_click(b):
        showPage("Edit Teacher")
    edit_teacher_btn.on_click(on_edit_teacher_click)

    def on_delete_teacher_click(b):
        showPage("Delete Teacher")
    delete_teacher_btn.on_click(on_delete_teacher_click)

    def on_add_course_click(b):
        showPage("Add Course")
    add_course_btn.on_click(on_add_course_click)

    def on_edit_course_click(b):
        showPage("Edit Course")
    edit_course_btn.on_click(on_edit_course_click)

    def on_delete_course_click(b):
        showPage("Delete Course")
    delete_course_btn.on_click(on_delete_course_click)


    # Initial table update / keep updating table
    update_tables()


    # Layout
    manager_db1HBox = widgets.HBox([logout_btn, refresh_btn])
    manager_db1VBox = widgets.VBox([title_label, teachers_label])
    manager_db2HBox = widgets.HBox([add_teacher_btn, edit_teacher_btn, delete_teacher_btn])
    manager_db2VBox = widgets.VBox([teachers_table, courses_label])
    manager_db3HBox = widgets.HBox([add_course_btn, edit_course_btn, delete_course_btn])
    manager_db3VBox = widgets.VBox([courses_table])


    display(widgets.VBox([manager_db1HBox, manager_db1VBox, manager_db2HBox, manager_db2VBox,
                          manager_db3HBox, manager_db3VBox, error_msg]))
################ Manager Dashboard Tab - End ################


##########################################################
################ Add Teacher Tab - Begin #################
##########################################################
with tb_all.output_to('Add Teacher'):

    # Label & Textbox
    Teacher_title_label = Label("Add New Teacher")

    F_Name_label = Label("First Name: ")
    F_Name_textbox = Text(placeholder= 'First_Name', disable = False)

    S_Name_label = Label("Surname Name: ")
    S_Name_textbox = Text(placeholder= 'Surname', disable = False)

    Email_label = Label("Email: ")
    Email_textbox = Text(placeholder = "Email", disable = False)

    PW_label = Label("Password: ")
    PW_textbox = Text(placeholder = "Password", disable = False)

    Status_label = Label("Status: ")
    Status_cmb = widgets.Dropdown(options=['Active', 'Not Active', 'On Leave', 'Training', 'Probation'], value = None)

    Tea_message_label = Label("")

    Tea_add_button = widgets.Button(description = 'Add')
    Tea_clear_button = widgets.Button(description = 'Clear')
    Tea_back_button = widgets.Button(description = 'Back')


    # Add Teacher
    def on_Tea_add_button_clicked(b):

      Tea_required_field = [
          F_Name_textbox.value.strip(),
          S_Name_textbox.value.strip(),
          Email_textbox.value.strip(),
          PW_textbox.value.strip(),
          Status_cmb.value
                            ]

      if not all(Tea_required_field):
          Tea_message_label.value = "Missing data: Please fill in all required fields."
          return  # Stop — don't add teacher

      # generate next Txxx ID
      c.execute("SELECT TER_ID FROM TEACHER ORDER BY TER_ID DESC")
      row = c.fetchone()
      if row is None:
            TER_ID = "T001"
      else:
            TER_last_id = row[0]
            TER_num = int(TER_last_id[1:]) + 1
            TER_ID = f"T{TER_num:03d}"

      c.execute(
          """Insert into TEACHER Values ('{}','{}', '{}', '{}', '{}','{}' )
          """.format(TER_ID, F_Name_textbox.value, S_Name_textbox.value, Email_textbox.value, PW_textbox.value, Status_cmb.value))
      c.execute("commit")
      Tea_add_button.disabled = True
      Tea_message_label.value = "Done!"
    Tea_add_button.on_click(on_Tea_add_button_clicked)

    # Clear
    def on_Tea_clear_button_clicked(b):
      F_Name_textbox.value = ""
      S_Name_textbox.value = ""
      Email_textbox.value = ""
      PW_textbox.value = ""
      Status_cmb.value = None
      Tea_add_button.disabled = False
      Tea_message_label.value = ""
    Tea_clear_button.on_click(on_Tea_clear_button_clicked)

    # Back
    def on_Tea_back_button_clicked(b):
      showPage("Manager Dashboard")
    Tea_back_button.on_click(on_Tea_back_button_clicked)

    # Display
    Tea_labelVBox = widgets.VBox([ F_Name_label, S_Name_label, Email_label, PW_label, Status_label])
    Tea_textboxVBox = widgets.VBox([ F_Name_textbox, S_Name_textbox, Email_textbox, PW_textbox, Status_cmb])
    Tea_line1HBox = widgets.HBox([Tea_labelVBox, Tea_textboxVBox])
    Tea_line2HBox = widgets.HBox([Tea_add_button, Tea_clear_button, Tea_back_button])


    display(widgets.VBox([Teacher_title_label, Tea_line1HBox, Tea_message_label, Tea_line2HBox]))
################ Add Teacher Tab - End ################



##########################################################
################ Edit Teacher Tab - Begin ################
##########################################################
with tb_all.output_to('Edit Teacher'):

    # Create widgets for teacher ID retrieval
    ET_title_label = Label("EDIT TEACHER INFORMATION") # Title label

    ET_teacher_id_input = Text(placeholder="Enter Teacher ID (e.g., T001)", description="Teacher ID:", layout={'width': '400px'})                      # Input field for teacher ID
    ET_retrieve_btn = widgets.Button(description="Retrieve Teacher", button_style='primary', layout={'width': '200px'})                                # Button to trigger teacher data retrieval

    spacer = Label("-----------------------------------------------------------------------------------------------------------------------------------") # Visual separator

    ET_current_header = Label("CURRENT INFORMATION", style={'font_weight': 'bold'}, layout={'width': '350px'})                                        # Header for current information section
    ET_edit_header = Label("MAKE CHANGES", style={'font_weight': 'bold'}, layout={'width': '350px'})                                                  # Header for editable fields section

    ET_lname_display = Text(description="Last Name:", disabled=True, layout={'width': '350px'})
    ET_lname_edit = Text(placeholder="Enter new last name", description="Last Name:", layout={'width': '350px'})

    ET_fname_display = Text(description="First Name:", disabled=True, layout={'width': '350px'})
    ET_fname_edit = Text(placeholder="Enter new first name", description="First Name:", layout={'width': '350px'})

    ET_email_display = Text(description="Email:", disabled=True, layout={'width': '350px'})
    ET_email_edit = Text(placeholder="Enter new email", description="Email:", layout={'width': '350px'})

    ET_status_display = Text(
        description="Status:",
        disabled=True,
        layout={'width': '350px'}
    )
    ET_status_edit = widgets.Dropdown(
        options=['Active', 'Not Active', 'On Leave', 'Training', 'Probation'],
        description="Status:",
        layout={'width': '350px'}
    )

    ET_password_label = Label("New Password:", layout={'width': '350px'})
    ET_password_edit = Text(placeholder="Leave blank to keep current password",
                           description="New Password:", layout={'width': '350px'})

    ET_message_label = Label("")                                                                                                                   # For displaying success/error messages to Manager

    # Buttons
    ET_clear_btn = widgets.Button(description="Clear Form", button_style='warning', layout={'width': '150px'})                                        #  Clear all form fields and reset the interface
    ET_edit_btn = widgets.Button(description="Update Teacher", button_style='success', layout={'width': '150px'})                                     #  Save all changes made to teacher information
    ET_back_btn = widgets.Button(description="Back to Dashboard", layout={'width': '150px'})                                                          #  Navigate back to main dashboard

    # Retrieve teacher function
    def on_ET_retrieve_btn_click(b):
        ET_teacher_id =  ET_teacher_id_input.value.strip()                                                                                             # Get and clean the teacher ID input, removing any extra spaces

        # Validation
        if not ET_teacher_id:
            ET_message_label.value = "ERROR: Please enter a Teacher ID"                                                                                  # Validation: Check if teacher ID is provided
            return # Stop if validation fails

        try:
            # Retrieve teacher from database
            c.execute("SELECT TER_ID, TER_LNAME, TER_FNAME, TER_EMAIL, TER_PW, TER_STATUS FROM TEACHER WHERE TER_ID = '" +  ET_teacher_id + "'")
            row = c.fetchone()                                                                                                                            #   Retrieve teacher data from database using teacher ID

            if row: # Check if teacher was found in database
                # Display current information
                ET_lname_display.value = row[1]
                ET_fname_display.value = row[2]
                ET_email_display.value = row[3]
                ET_status_display.value = row[5]

                # Pre-fill edit fields with current values
                ET_lname_edit.value = row[1]
                ET_fname_edit.value = row[2]
                ET_email_edit.value = row[3]
                ET_status_edit.value = row[5]
                ET_password_edit.value = ''

                ET_message_label.value = f"SUCCESS: Teacher { ET_teacher_id} retrieved successfully"     # Success message confirming retrieval
            else:
                ET_message_label.value = f"ERROR: No teacher found with ID: { ET_teacher_id}"            # Error message for non-existent teacher ID

                # Clear all fields
                ET_lname_display.value = ""
                ET_fname_display.value = ""
                ET_email_display.value = ""
                ET_status_display.value = "Active"
                ET_lname_edit.value = ""
                ET_fname_edit.value = ""
                ET_email_edit.value = ""
                ET_status_edit.value = "Active"
                ET_password_edit.value = ""

        except Exception as e:                                              # Handle any database errors that occured
            ET_message_label.value = f"ERROR: Database error - {str(e)}"
    ET_retrieve_btn.on_click(on_ET_retrieve_btn_click)

    # Clear function
    def on_clear_click(b):
        ET_teacher_id_input.value = ""
        ET_lname_display.value = ""
        ET_fname_display.value = ""
        ET_email_display.value = ""
        ET_status_display.value = "Active"
        ET_lname_edit.value = ""
        ET_fname_edit.value = ""
        ET_email_edit.value = ""
        ET_status_edit.value = "Active"
        ET_password_edit.value = ""
        ET_message_label.value = ""
    ET_clear_btn.on_click(on_clear_click)

    # Edit teacher function
    def on_ET_edit_btn_click(b):
        ET_teacher_id =  ET_teacher_id_input.value.strip()

        # Validation
        if not ET_teacher_id:
            ET_message_label.value = "ERROR: Please enter a Teacher ID first"
            return

        # Validate required fields
        if not all([ET_lname_edit.value, ET_fname_edit.value, ET_email_edit.value]):
            ET_message_label.value = "ERROR: Please fill in all required fields (Last Name, First Name, Email)"
            return

        try:
            # Check if teacher exists
            c.execute("SELECT TER_ID FROM TEACHER WHERE TER_ID = '" + ET_teacher_id + "'")
            if not c.fetchone():
                ET_message_label.value = f"ERROR: No teacher found with ID: {ET_teacher_id}"
                return

            # Update teacher in database
            if ET_password_edit.value:
                # Update all fields including password
                c.execute(
                    """UPDATE TEACHER SET TER_LNAME = '{}', TER_FNAME = '{}', TER_EMAIL = '{}', TER_STATUS = '{}', TER_PW = '{}' WHERE TER_ID = '{}'""".format(
                        ET_lname_edit.value, ET_fname_edit.value, ET_email_edit.value, ET_status_edit.value, ET_password_edit.value, ET_teacher_id
                    )
                )
            else:
                # Update all fields except password
                c.execute(
                    """UPDATE TEACHER SET TER_LNAME = '{}', TER_FNAME = '{}', TER_EMAIL = '{}', TER_STATUS = '{}' WHERE TER_ID = '{}'""".format(
                        ET_lname_edit.value, ET_fname_edit.value, ET_email_edit.value, ET_status_edit.value, ET_teacher_id
                    )
                )

            c.execute("COMMIT")

            # Update display fields with new values
            ET_lname_display.value = ET_lname_edit.value
            ET_fname_display.value = ET_fname_edit.value
            ET_email_display.value = ET_email_edit.value
            ET_status_display.value = ET_status_edit.value

            ET_message_label.value = f"SUCCESS: Teacher {ET_teacher_id} was updated successfully"

            # Clear password field
            ET_password_edit.value = ""

        except Exception as e:  # Handle any errors that occured
            ET_message_label.value = f"ERROR: Failed to update teacher - {str(e)}"
    ET_edit_btn.on_click(on_ET_edit_btn_click)

    # Back to dashboard function
    def on_ET_back_btn_click(b):
        showPage("Manager Dashboard")
    ET_back_btn.on_click(on_ET_back_btn_click)

    # Layout
    TeaEdit_line1HBox = widgets.HBox([ET_teacher_id_input, ET_retrieve_btn]) # Horizontal layout: Teacher ID input and retrieve button side by side
    TeaEdit_line1VBox = widgets.VBox([ET_current_header, ET_lname_display, ET_fname_display, ET_email_display, ET_status_display])  # Vertical layout: Current information display fields stacked vertically
    TeaEdit_line2VBox = widgets.VBox([ET_edit_header, ET_lname_edit, ET_fname_edit, ET_email_edit, ET_status_edit, ET_password_edit])  # Vertical layout: Editable fields stacked vertically for making changes
    TeaEdit_line2HBox = widgets.HBox([ET_clear_btn, ET_edit_btn, ET_back_btn]) # Horizontal layout: Action buttons arranged side by side
    TeaEdit_line3HBox = widgets.HBox([TeaEdit_line1VBox, TeaEdit_line2VBox])  # Horizontal layout: Current info and edit sections displayed side by side for comparison

    display(widgets.VBox([
        ET_title_label,
        TeaEdit_line1HBox,
        spacer,
        TeaEdit_line3HBox,
        TeaEdit_line2HBox,
        ET_message_label
    ]))  # Final vertical arrangement of all UI components
################ Edit Teacher Tab - End ################


##########################################################
################ Delete Teacher Tab - Begin ################
##########################################################
with tb_all.output_to('Delete Teacher'):
    # create labels and inputs
    title_label = Label("Delete Teacher")

    DEL_id_label = Label("Teacher ID: ")
    DEL_id_textbox = Text(placeholder="Teacher ID")

    DEL_spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")

    DEL_L_name_textbox = Text(placeholder="Name will appear here", description="Last Name:", disabled=True)
    DEL_F_name_textbox = Text(placeholder="Name will appear here", description="First Name:", disabled=True)
    DEL_email_textbox = Text(placeholder="Email will appear here", description="Email:", disabled=True)

    status = widgets.Dropdown(
        options=['Active', 'Not Active', 'On Leave', 'Training', 'Probation'],
        value='Active',
        description="Status:"
    )

    DEL_Tea_message_label = Label("")

    # create 4 buttons
    DEL_retrieve_button = widgets.Button(description="Retrieve")
    DEL_delete_button = widgets.Button(description="Delete")
    DEL_clear_button = widgets.Button(description="Clear")
    DEL_back_button = widgets.Button(description="Back")

    def on_DEL_retrieve_button_clicked(b):
        c.execute("SELECT * FROM TEACHER WHERE TER_ID = :1", (DEL_id_textbox.value,))
        for row in c:
            DEL_L_name_textbox.value = row[1]
            DEL_F_name_textbox.value = row[2]
            DEL_email_textbox.value = row[3]
            status.value = row[5]

    DEL_retrieve_button.on_click(on_DEL_retrieve_button_clicked)

    def on_DEL_delete_button_clicked(b):
        teacher_id = DEL_id_textbox.value.strip()

        if not teacher_id:
            DEL_Tea_message_label.value = "Please enter a Teacher ID first."
            return

        try:
            # Check if teacher exists in COURSES table (has dependent records)
            c.execute("SELECT COUNT(*) FROM COURSES WHERE TEACHER_TER_ID = :1", (teacher_id,))
            course_count = c.fetchone()[0]    # Course ID locates at position [0] in database

            if course_count > 0:
                DEL_Tea_message_label.value = f"Cannot delete teacher {teacher_id}. This teacher is assigned to {course_count} course(s). Please reassign or delete those courses first."
                return

            # If no dependent courses, proceed with deletion
            c.execute("DELETE FROM TEACHER WHERE TER_ID = :1", (teacher_id,))     # Run a SQL query to find a teacher by ID, using a SAFE placeholder :1 (First Parameter) and passing the actual value separately
            conn.commit()  # Use conn.commit() instead of c.execute("commit") since c.execute("commit") is to run SQL instead of saving changes
            DEL_Tea_message_label.value = "Delete Success!"

            # DO NOT clear the form - keep the success message visible
            # The user can see the success message until they click "Back to Dashboard" or "Clear"

        except Exception as e:
            DEL_Tea_message_label.value = f"Error: {e}"

    DEL_delete_button.on_click(on_DEL_delete_button_clicked)

    def on_DEL_back_button_clicked(b):
        showPage("Manager Dashboard")
    DEL_back_button.on_click(on_DEL_back_button_clicked)

    # display the objects
    DEL_line1HBox = widgets.HBox([title_label])
    DEL_line2HBox = widgets.HBox([DEL_id_label, DEL_id_textbox, DEL_retrieve_button])
    DEL_line3HBox = widgets.HBox([DEL_spacer])
    DEL_textboxVBox = widgets.VBox([DEL_L_name_textbox, DEL_F_name_textbox, DEL_email_textbox, status])
    DEL_line4HBox = widgets.HBox([DEL_textboxVBox])
    DEL_line5HBox = widgets.HBox([DEL_delete_button, DEL_clear_button])
    DEL_line6HBox = widgets.HBox([DEL_back_button])

    display(widgets.VBox([DEL_line1HBox, DEL_line2HBox, DEL_line3HBox, DEL_line4HBox, DEL_Tea_message_label, DEL_line5HBox, DEL_line6HBox]))
################ Delete Teacher Tab - End ################


##########################################################
################ Add Course Tab - Begin ################
##########################################################
with tb_all.output_to('Add Course'):

# Label & Textboxs
  COU_title_label = Label("Add New Course")

  #COU_Name_label = Label("Course Name: ")
  COU_Name_textbox = Text(description = 'Name: ', placeholder= 'Input course name', disable = False)

  #COU_DESC_label = Label("Decription: ")
  COU_DESC_textbox = widgets.Textarea(description = 'Decription: ', placeholder= 'Input course description (max 500 characters)', layout={'width': '500px', 'height':'100px'})

  #COU_TER_label = Label("Teacher: ")
  COU_TER_textbox = Text(description = "Teacher: ",placeholder = "", disable = False)

  #COU_TER_ID_label = Label("Teacher ID: ")
  COU_TER_ID_textbox = Text(description = 'Teacher ID: ', placeholder = "", disable = False)

  #COU_CAT_label = Label("Category: ")
  COU_CAT_cmb = widgets.Dropdown(description = 'Category: ', options=['Art', 'Math', 'Science', 'Music', 'Dance', 'Sports',
                'Language', 'Coding', 'Cooking', 'Drama', 'Yoga', 'Robotics'], value= None)

  #COU_AGE_MIN_label = Label("Min Age: ")
  COU_AGE_MIN_intbox = IntText(description= "Min Age: ", disable = False)

  #COU_AGE_MAX_label = Label("Max Age: ")
  COU_AGE_MAX_intbox = IntText(description= "Max Age: ", disable = False)

  #COU_QUOTA_label = Label("Quota: ")
  COU_QUOTA_intbox = IntText(description = "Quota: ", disable = False)

  #COU_STATUS_label = Label("Status: ")
  COU_STATUS_cmb = widgets.Dropdown(description = 'Status: ', options=['Active', 'Cancelled', 'Postponed', 'Full', 'Upcoming'],
        value='Active')

  #COU_DATE_label = Label("Course Date: ")
  COU_DATE_datepicker = DatePicker(description = 'Course Date: ', placeholder="", disable = False)

  #COU_ENT_DATE_label = Label("Enrolment Date: ")
  COU_ENT_DATE_datepicker = DatePicker(description = "Enrolment Date: ", placeholder="", disable = False)

  #COU_REMARKS_label = Label("Remarks: ")
  COU_REMARKS_textbox = widgets.Textarea(description = "Remarks: ", placeholder = "Optional (max 300 characters)", layout={'width': '500px', 'height':'100px'} )

  COU_message_label = Label("")

# Create buttons

  COU_add_button = widgets.Button(description= "Add Course")
  COU_cancel_button = widgets.Button(description= "Clear")
  COU_back_button = widgets.Button(description= "Back")

# Event Handler

  def on_COU_add_button_clicked(b):
    COU_required_field = [
        COU_Name_textbox.value.strip(),
        COU_DESC_textbox.value.strip(),
        COU_QUOTA_intbox.value,
        COU_DATE_datepicker.value,
        COU_ENT_DATE_datepicker.value,
        COU_STATUS_cmb.value,
        COU_CAT_cmb.value,
        COU_TER_ID_textbox.value.strip()

       ]

    # validate if all fields are filled
    if not all(COU_required_field):
        COU_message_label.value = "Missing data: Please fill in all the fields."
        return  # Stop execution — don't add course

    # age valiadation
    if COU_AGE_MIN_intbox.value > COU_AGE_MAX_intbox.value:
        COU_message_label.value = "ERROR: Min age > Max age !"
        return # Stop execution - mix age > max age

    # generate next COUxxx ID
    c.execute("SELECT COU_ID FROM COURSES ORDER BY COU_ID DESC")
    row = c.fetchone()
    if row is None:
          COU_ID = "C001"
    else:
        COU_last_id = row[0]
        COU_num = int(COU_last_id[1:]) + 1
        COU_ID = f"C{COU_num:03d}"

    c.execute(
        """Insert into COURSES Values (
          '{}',
          '{}',
          '{}',
          {},
          TO_DATE('{}','YYYY-MM-DD'),
          TO_DATE('{}','YYYY-MM-DD'),
          '{}',
          {},
          {},
          '{}',
          '{}',
          '{}'
          )

        """.format(
            COU_ID,
            COU_Name_textbox.value,
            COU_DESC_textbox.value,
            COU_QUOTA_intbox.value,
            COU_DATE_datepicker.value.strftime('%Y-%m-%d'),
            COU_ENT_DATE_datepicker.value.strftime('%Y-%m-%d'),
            COU_STATUS_cmb.value,
            COU_AGE_MAX_intbox.value,
            COU_AGE_MIN_intbox.value,
            COU_REMARKS_textbox.value,
            COU_CAT_cmb.value,
            COU_TER_ID_textbox.value
            )
        )
    c.execute("commit")
    COU_add_button.disabled = True
    COU_message_label.value = "Done !"
  COU_add_button.on_click(on_COU_add_button_clicked)

  # Clear
  def on_COU_clear_button_clicked(b):
    COU_Name_textbox.value = ""
    COU_DESC_textbox.value = ""
    COU_TER_textbox.value = ""
    COU_TER_ID_textbox.value = ""
    COU_CAT_cmb.value = None
    COU_AGE_MIN_intbox.value = 0
    COU_AGE_MAX_intbox.value = 0
    COU_QUOTA_intbox.value = 0
    COU_STATUS_cmb.value = None
    COU_DATE_datepicker.value = None
    COU_ENT_DATE_datepicker.value = None
    COU_START_TIME_cmb = None
    COU_END_TIME_cmb = None
    COU_REMARKS_textbox.value = ""
    COU_message_label.value = ""

    COU_add_button.disabled = False
  COU_cancel_button.on_click(on_COU_clear_button_clicked)

  # Back
  def on_COU_back_button_clicked(b):
    showPage("Manager Dashboard")
  COU_back_button.on_click(on_COU_back_button_clicked)

# Display

  display(widgets.VBox([
      widgets.HBox([COU_title_label]),
      widgets.HBox([COU_Name_textbox]),
      widgets.HBox([COU_DESC_textbox]),
      widgets.HBox([COU_TER_textbox, COU_TER_ID_textbox]),
      widgets.HBox([COU_AGE_MIN_intbox, COU_CAT_cmb]),
      widgets.HBox([COU_AGE_MAX_intbox,COU_QUOTA_intbox]),
      widgets.HBox([COU_DATE_datepicker, COU_STATUS_cmb]),
      widgets.HBox([COU_ENT_DATE_datepicker]),
      widgets.HBox([COU_REMARKS_textbox]),
      widgets.HBox([COU_message_label]),
      widgets.HBox([COU_add_button, COU_back_button, COU_cancel_button])
  ]))
################ Add Course Tab - End ################


##########################################################
################ Edit Course Tab - Begin ################
##########################################################
with tb_all.output_to('Edit Course'):

    # Create widgets for course ID retrieval
    title_label = Label("EDIT COURSE INFORMATION")

    course_id_input = Text(placeholder="Enter Course ID (e.g., C001)", description="Course ID:", layout={'width': '400px'})      # Input field for course ID
    retrieve_btn = widgets.Button(description="Retrieve Course", button_style='primary', layout={'width': '200px'})             # Button to trigger course data retrieval

    spacer = Label("-----------------------------------------------------------------------------------------------------------------------------------")    # Visual separator

    current_header = Label("CURRENT INFORMATION", style={'font_weight': 'bold'}, layout={'width': '350px'})
    edit_header = Label("MAKE CHANGES", style={'font_weight': 'bold'}, layout={'width': '350px'})

    course_name_display = Text(description="Course Name:", disabled=True, layout={'width': '350px'})
    course_name_edit = Text(placeholder="Enter new course name", description="Course Name:", layout={'width': '350px'})

    description_display = widgets.Textarea(description="Description:", disabled=True, layout={'width': '350px', 'height': '80px'})      # Textarea for displaying current course descript
    description_edit = widgets.Textarea(placeholder="Enter new description", description="Description:", layout={'width': '350px', 'height': '80px'})   # Editable textarea for updating course descript

    teacher_display = widgets.Dropdown(description="Teacher:", disabled=True, layout={'width': '350px'})  # Dropdown for displaying current assigned teacher
    teacher_edit = widgets.Dropdown(description="Teacher:", layout={'width': '350px'})     # Editable dropdown for selecting a new teacher from available options

    category_display = widgets.Dropdown(
        options=['Art', 'Math', 'Science', 'Music', 'Dance', 'Sports', 'Language', 'Coding', 'Cooking', 'Drama', 'Yoga', 'Robotics'],
        description="Category:",
        disabled=True,
        layout={'width': '350px'}
    )
    category_edit = widgets.Dropdown(
        options=['Art', 'Math', 'Science', 'Music', 'Dance', 'Sports', 'Language', 'Coding', 'Cooking', 'Drama', 'Yoga', 'Robotics'],
        description="Category:",
        layout={'width': '350px'}
    )

    age_display = widgets.HBox([
        IntText(description="Min Age:", disabled=True, layout={'width': '160px'}),  # Horizontal container for displaying current age range
        IntText(description="Max Age:", disabled=True, layout={'width': '160px'})
    ], layout={'width': '350px'})
    age_edit = widgets.HBox([
        IntText(description="Min Age:", min=4, max=12, layout={'width': '160px'}), # Horizontal container for editing age range
        IntText(description="Max Age:", min=4, max=12, layout={'width': '160px'}) #with validation constraints
    ], layout={'width': '350px'})

    quota_display = IntText(description="Quota:", disabled=True, layout={'width': '350px'})
    quota_edit = IntText(description="Quota:", min=1, max=99, layout={'width': '350px'})
    quota_row = widgets.HBox([quota_display, quota_edit])

    status_display = widgets.Dropdown(
        options=['Active', 'Cancelled', 'Postponed', 'Full', 'Upcoming'],
        description="Status:",
        disabled=True,
        layout={'width': '350px'}
    )
    status_edit = widgets.Dropdown(
        options=['Active', 'Cancelled', 'Postponed', 'Full', 'Upcoming'],
        description="Status:",
        layout={'width': '350px'}
    )

    course_date_display = DatePicker(description="Course Date:", disabled=True, layout={'width': '350px'})
    course_date_edit = DatePicker(description="Course Date:", layout={'width': '350px'})

    deadline_display = DatePicker(description="Enrollment Deadline:", disabled=True, layout={'width': '350px'})
    deadline_edit = DatePicker(description="Enrollment Deadline:", layout={'width': '350px'})

    remarks_display = widgets.Textarea(description="Remarks:", disabled=True, layout={'width': '350px', 'height': '60px'})
    remarks_edit = widgets.Textarea(placeholder="Enter new remarks", description="Remarks:", layout={'width': '350px', 'height': '60px'})

    message_label = Label("")  # Displaying success/error messages to user

    # Buttons
    clear_btn = widgets.Button(description="Clear Form", button_style='warning', layout={'width': '150px'})
    edit_btn = widgets.Button(description="Update Course", button_style='success', layout={'width': '150px'})
    back_btn = widgets.Button(description="Back to Dashboard", layout={'width': '150px'})

    # Update teacher dropdown from database
    def update_teacher_dropdown():
        try:
            c.execute("SELECT TER_ID, TER_FNAME, TER_LNAME FROM TEACHER")  # Query database to get all available teachers
            teachers = c.fetchall()
            # Add a blank option at the beginning
            options = [""] + [f"{teacher[1]} {teacher[2]} ({teacher[0]})" for teacher in teachers] # "FirstName LastName (T001)"
            teacher_display.options = options
            teacher_edit.options = options
        except Exception as e:
            message_label.value = f"ERROR: Failed to load teachers - {str(e)}"

    # Age validation
    def validate_age_range():
        if age_edit.children[0].value is not None and age_edit.children[1].value is not None and age_edit.children[0].value > age_edit.children[1].value:
            age_edit.children[0].value = age_edit.children[1].value   # Ensure min age doesn't exceed max age

    # Retrieve course function
    def on_retrieve_click(b):
        course_id = course_id_input.value.strip()

        # Validation
        if not course_id:
            message_label.value = "ERROR: Please enter a Course ID"
            return

        try:
            # Retrieve course from database
            c.execute("SELECT COU_ID, COU_NAME, COU_DESC, COU_QUOTA, COU_DATE, COU_ENROLLMENT_DEADLINE, COU_STATUS, COU_AGE_MAX, COU_AGE_MIN, COU_REMARKS, COU_CATEGORY, TEACHER_TER_ID FROM COURSES WHERE COU_ID = '" + course_id + "'")
            row = c.fetchone()

            if row:
                # Display current information
                course_name_display.value = row[1]
                description_display.value = row[2]
                quota_display.value = row[3]

                # Handle date conversion - extract date part only
                course_date_db = str(row[4]).split()[0]  # Get only date part
                deadline_db = str(row[5]).split()[0]     # Get only date part

                course_date_display.value = datetime.strptime(course_date_db, "%Y-%m-%d").date()
                deadline_display.value = datetime.strptime(deadline_db, "%Y-%m-%d").date()

                status_display.value = row[6]  # COU_STATUS
                age_display.children[1].value = row[7] if row[7] else 8
                age_display.children[0].value = row[8] if row[8] else 4
                remarks_display.value = row[9] if row[9] else ""
                category_display.value = row[10]

                # Pre-fill edit fields with current values
                course_name_edit.value = row[1]
                description_edit.value = row[2]
                quota_edit.value = row[3]
                course_date_edit.value = datetime.strptime(course_date_db, "%Y-%m-%d").date()
                deadline_edit.value = datetime.strptime(deadline_db, "%Y-%m-%d").date()
                status_edit.value = row[6]
                age_edit.children[1].value = row[7] if row[7] else 8
                age_edit.children[0].value = row[8] if row[8] else 4
                remarks_edit.value = row[9] if row[9] else ""
                category_edit.value = row[10]

                # Set teacher dropdown
                update_teacher_dropdown()
                current_teacher_option = next((opt for opt in teacher_edit.options if row[11] in opt), None)
                if current_teacher_option:
                    teacher_display.value = current_teacher_option
                    teacher_edit.value = current_teacher_option

                message_label.value = f"SUCCESS: Course {course_id} retrieved successfully"
            else:
                message_label.value = f"ERROR: No course found with ID: {course_id}"

                # Clear all fields
                course_name_display.value = ""
                description_display.value = ""
                teacher_display.value = None # Set to None
                category_display.value = "Art"
                age_display.children[0].value = 4
                age_display.children[1].value = 8
                quota_display.value = 15
                status_display.value = "Active"
                course_date_display.value = date.today()
                deadline_display.value = date.today()
                remarks_display.value = ""

                course_name_edit.value = ""
                description_edit.value = ""
                teacher_edit.value = None # Set to None
                category_edit.value = "Art"
                age_edit.children[0].value = 4
                age_edit.children[1].value = 8
                quota_edit.value = 15
                status_edit.value = "Active"
                course_date_edit.value = date.today()
                deadline_edit.value = date.today()
                remarks_edit.value = ""

        except Exception as e:
            message_label.value = f"ERROR: Database error - {str(e)}"
    retrieve_btn.on_click(on_retrieve_click)

    # Clear function
    def on_clear_click(b):
        course_id_input.value = ""
        course_name_display.value = ""
        description_display.value = ""
        teacher_display.value = None
        category_display.value = "Art"
        age_display.children[0].value = 4
        age_display.children[1].value = 8
        quota_display.value = 15
        status_display.value = "Active"
        course_date_display.value = date.today()
        deadline_display.value = date.today()
        remarks_display.value = ""

        course_name_edit.value = ""
        description_edit.value = ""
        teacher_edit.value = None
        category_edit.value = "Art"
        age_edit.children[0].value = 4
        age_edit.children[1].value = 8
        quota_edit.value = 15
        status_edit.value = "Active"
        course_date_edit.value = date.today()
        deadline_edit.value = date.today()
        remarks_edit.value = ""

        message_label.value = ""
    clear_btn.on_click(on_clear_click)

    # Edit course function
    def on_edit_click(b):
        course_id = course_id_input.value.strip()

        # Validation
        if not course_id:
            message_label.value = "ERROR: Please enter a Course ID first"
            return

        # Validate required fields
        if not all([course_name_edit.value, description_edit.value, teacher_edit.value,
                   age_edit.children[0].value, age_edit.children[1].value, quota_edit.value,
                   course_date_edit.value, deadline_edit.value]):
            message_label.value = "ERROR: Please fill in all required fields"
            return

        # Validate field lengths
        if len(course_name_edit.value) > 50:
            message_label.value = "ERROR: Course name must be 50 characters or less"
            return

        if len(description_edit.value) > 500:
            message_label.value = "ERROR: Description must be 500 characters or less"
            return

        if len(remarks_edit.value) > 300:
            message_label.value = "ERROR: Remarks must be 300 characters or less"
            return

        try:
            # Check if course exists
            c.execute("SELECT COU_ID FROM COURSES WHERE COU_ID = '" + course_id + "'")
            if not c.fetchone():
                message_label.value = f"ERROR: No course found with ID: {course_id}"
                return

            # Extract teacher ID from dropdown
            teacher_text = teacher_edit.value
            teacher_id = teacher_text.split('(')[-1].replace(')', '')

            # Clean dates to remove time components
            course_date_str = course_date_edit.value.strftime("%Y-%m-%d")
            deadline_str = deadline_edit.value.strftime("%Y-%m-%d")

            # Update course in database
            c.execute(
                """UPDATE COURSES SET COU_NAME = '{}', COU_DESC = '{}', COU_QUOTA = {}, COU_DATE = TO_DATE('{}', 'YYYY-MM-DD'),
                COU_ENROLLMENT_DEADLINE = TO_DATE('{}', 'YYYY-MM-DD'), COU_STATUS = '{}',
                COU_AGE_MAX = {}, COU_AGE_MIN = {}, COU_REMARKS = '{}', COU_CATEGORY = '{}', TEACHER_TER_ID = '{}'
                WHERE COU_ID = '{}'""".format(
                    course_name_edit.value.replace("'", "''"),
                    description_edit.value.replace("'", "''"),
                    quota_edit.value,
                    course_date_str,
                    deadline_str,
                    status_edit.value,
                    age_edit.children[1].value,
                    age_edit.children[0].value,
                    remarks_edit.value.replace("'", "''"),
                    category_edit.value,
                    teacher_id,
                    course_id
                )
            )

            c.execute("COMMIT")

            # Update display fields with new values
            course_name_display.value = course_name_edit.value
            description_display.value = description_edit.value
            teacher_display.value = teacher_edit.value
            category_display.value = category_edit.value
            age_display.children[0].value = age_edit.children[0].value
            age_display.children[1].value = age_edit.children[1].value
            quota_display.value = quota_edit.value
            status_display.value = status_edit.value
            course_date_display.value = course_date_edit.value
            deadline_display.value = deadline_edit.value
            remarks_display.value = remarks_edit.value

            message_label.value = f"SUCCESS: Course {course_id} was updated successfully"

        except Exception as e:
            message_label.value = f"ERROR: Failed to update course - {str(e)}"
    edit_btn.on_click(on_edit_click)

    # Back to dashboard function
    def on_back_click(b):
        showPage("Manager Dashboard")
    back_btn.on_click(on_back_click)

    # Set up age validation
    age_edit.children[0].observe(lambda change: validate_age_range(), names='value')
    age_edit.children[1].observe(lambda change: validate_age_range(), names='value')

    # Initial setup
    update_teacher_dropdown()

    # Layout
    COUEdit_line1HBox = widgets.HBox([course_id_input, retrieve_btn])
    COUEdit_line1VBox = widgets.VBox([current_header, course_name_display, description_display, teacher_display, category_display, age_display, quota_display, status_display, course_date_display, deadline_display, remarks_display])
    COUEdit_line2VBox = widgets.VBox([edit_header, course_name_edit, description_edit, teacher_edit, category_edit, age_edit, quota_edit, status_edit, course_date_edit, deadline_edit, remarks_edit])
    COUEdit_line2HBox = widgets.HBox([clear_btn, edit_btn, back_btn])
    COUEdit_line3HBox = widgets.HBox([COUEdit_line1VBox, COUEdit_line2VBox])

    display(widgets.VBox([title_label, COUEdit_line1HBox, spacer, COUEdit_line3HBox, COUEdit_line2HBox]))
################ Edit Course Tab - End ################


##########################################################
################ Delete Course Tab - Begin ################
##########################################################

with tb_all.output_to('Delete Course'):

# create 6 label and 5 textboxes
  DELC_id_label = Label("Course ID: ")
  DELC_id_textbox = Text(value='', placeholder='Course ID (e.g. C001)', disabled=False)

  DELC_spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")

  DELC_name_label = Label("Course Name: ")
  DELC_name_textbox = Text(placeholder='Course Name', description='Course Name:', disabled=False)

  DELC_desc_label = Label("Course Description: ")
  DELC_desc_textbox = Text(description='Course Description:', disabled=False)

  DELC_quota_label = Label("Quota: ")
  DELC_quota_intbox = IntText(description='Quota:', disabled=False)

  DELC_cdate_label = Label("Course Date:")
  DELC_cdate_datepicker = DatePicker(description='Course Date:', disabled=False)

  DELC_edate_label = Label("Enrollment Deadline:")
  DELC_edate_datepicker = DatePicker(description='Enrollment Deadline:', disabled=False)

  DELC_status_label = Label("Status:") # Added label for Status
  DELC_status_dropdown = widgets.Dropdown( # Changed to Dropdown
        options=['Active', 'Not Active'], # Assuming these are the status options
        description="Status:",
        disabled=False # Enabled for display
    )

  DELC_agemin_label = Label("Age Min:")
  DELC_agemin_intbox = IntText(description='Age Min:', disabled=False)
  DELC_agemax_label = Label("Age Max:")
  DELC_agemax_intbox = IntText(description='Age Max:', disabled=False)

  DELC_remarks_label = Label("Remarks:")
  DELC_remarks_textbox = Text(description='Remarks:', disabled=False)

  DELC_category_label = Label("Category:")
  DELC_category_textbox = Text(description='Category:', disabled=False)

  DELC_tea_message_label = Label("")


# create 4 buttons
  DELC_retrieve_button = widgets.Button(description="Retrieve")
  DELC_delete_button = widgets.Button(description="Delete")
  DELC_clear_button = widgets.Button(description="Clear")
  DELC_back_button = widgets.Button(description="Back")

# define the event handlers for the three buttons when they are being clicked
  def on_DELC_retrieve_button_clicked(b):
    c.execute("SELECT * FROM COURSES WHERE cou_id = '" + DELC_id_textbox.value + "'")
    for row in c:
      DELC_name_textbox.value = row[1]
      DELC_desc_textbox.value = row[2]
      DELC_quota_intbox.value = row[3]
      DELC_cdate_datepicker.value = row[4]
      DELC_edate_datepicker.value = row[5]
      DELC_status_dropdown.value = row[6] # Assuming status is at index 6 and setting dropdown value
      DELC_agemax_intbox.value = row[7] # Assuming age min is at index 7
      DELC_agemin_intbox.value = row[8] # Assuming age max is at index 8
      DELC_remarks_textbox.value = row[9] # Assuming remarks is at index 9
      DELC_category_textbox.value = row[10] # Assuming category is at index 10


  DELC_retrieve_button.on_click(on_DELC_retrieve_button_clicked)

  def on_DELC_clear_button_clicked(b):
    DELC_id_textbox.value = ""
    DELC_name_textbox.value = ""
    DELC_desc_textbox.value = ""
    DELC_quota_intbox.value = 0
    DELC_cdate_datepicker.value = None
    DELC_edate_datepicker.value = None
    DELC_status_dropdown.value = 'Active' # Reset dropdown to default
    DELC_agemin_intbox.value = 0 # Changed to 0 for IntText
    DELC_agemax_intbox.value = 0 # Changed to 0 for IntText
    DELC_remarks_textbox.value = "" # Clear Remarks
    DELC_category_textbox.value = "" # Clear Category
    DELC_tea_message_label.value = ""

  DELC_clear_button.on_click(on_DELC_clear_button_clicked)

  def on_DELC_delete_button_clicked(b):
    c.execute(
    """
          DELETE FROM COURSES
          WHERE cou_id = '{}'
    """.format(DELC_id_textbox.value)
    )
    c.execute("commit")
    DELC_tea_message_label.value = "Delete Success!"
  DELC_delete_button.on_click(on_DELC_delete_button_clicked)

  def on_DELC_back_button_clicked(b):
    showPage("Manager Dashboard")
  DELC_back_button.on_click(on_DELC_back_button_clicked)

# display the objects
  DELC_line1HBox = widgets.HBox([DELC_id_textbox, DELC_retrieve_button])
  DELC_line2HBox = widgets.HBox([DELC_spacer])
  DELC_textboxVBox = widgets.VBox([DELC_name_textbox, DELC_desc_textbox, DELC_quota_intbox, DELC_cdate_datepicker, DELC_edate_datepicker, DELC_status_dropdown, DELC_agemax_intbox, DELC_agemin_intbox, DELC_remarks_textbox, DELC_category_textbox]) # Swapped order of agemax and agemin
  DELC_line3HBox = widgets.HBox([DELC_textboxVBox])
  DELC_line4HBox = widgets.HBox([DELC_delete_button, DELC_clear_button])
  DELC_line5HBox = widgets.HBox([DELC_back_button])

  display(widgets.VBox([DELC_line1HBox, DELC_line2HBox, DELC_line3HBox, DELC_tea_message_label, DELC_line4HBox, DELC_line5HBox]))
################ Delete Course Tab - End ################


showPage("Home Page")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

We have setup two accounts:

(1)
- MEM_ID: M001
- MEM_PW: 123456
- MEM_FNAME: Issac
- MEM_LNAME: Wong
- MEM_DOB: 03/01/2005
- K_FNAME: Tom
- K_LNAME: Wong
- K_DOB: 05/01/2010
- MEM_SECURITY_Q: What is your favorite fruit?
- MEM_SECURITY_ANS: Apple
- MEM_TEL: 94338096
- MEM_ADD: The Peak
- MEM_EMAIL: tlwongay@connect.ust.hk
- MEM_REG_DATE: 29/10/2025


(2)
- MEM_ID: M002
- MEM_PW: 1235
- MEM_FNAME: Chris
- MEM_LNAME: BB
- MEM_DOB: 01/01/2005
- K_FNAME: Samuel
- K_LNAME: BB
- K_DOB: 01/01/2010
- MEM_SECURITY_Q: What is your favorite fruit?
- MEM_SECURITY_ANS: Orange
- MEM_TEL: 999
- MEM_ADD: HKUST
- MEM_EMAIL: ust.com
- MEM_REG_DATE: 29/10/2025

In [ ]:
import streamlit as st
import pandas as pd

# Sample data - replace this with your actual data source (e.g., from a database or CSV)
teachers = pd.DataFrame({
    'name': ['Teacher1', 'Teacher2', 'Teacher3', 'Teacher4', 'Teacher5'],
    'subject': ['Math', 'Science', 'English', 'History', 'Art'],
    'details': ['Details about Teacher1: Age 35, Experience 10 years.',
                'Details about Teacher2: Age 42, Experience 15 years.',
                'Details about Teacher3: Age 28, Experience 5 years.',
                'Details about Teacher4: Age 50, Experience 25 years.',
                'Details about Teacher5: Age 31, Experience 7 years.']
})

st.title('View Teacher Page')

# Retrieve the whole rows of the first five teachers
first_five = teachers.head(5)

# Display the first five teachers in a table-like format
st.subheader('First Five Teachers')
for i, row in first_five.iterrows():
    cols = st.columns([2, 2, 1])  # Columns for name, subject, and optional button
    with cols[0]:
        st.write(row['name'])
    with cols[1]:
        st.write(row['subject'])
    if i == 2:  # 0-based index, so 3rd row is index 2
        with cols[2]:
            if st.button('View', key=f'view_{i}'):
                st.subheader(f'More Details for {row["name"]}')
                st.write(row['details'])  # Display additional details when button is clicked

# Optionally, print the raw rows to console for debugging (not shown in the app)
print("Retrieved rows of first five teachers:")
print(first_five)

In [ ]:

##########################################################
################ Delete Course Tab - Begin ################
##########################################################

with tb_all.output_to('Delete Course'):

# create 6 label and 5 textboxes
  DELC_id_label = Label("Course ID")
  DELC_id_textbox = Text(value='', placeholder='Course ID (e.g. C001)', disabled=False)

  DELC_spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")

  DELC_name_label = Label("Course Name: ")
  DELC_name_textbox = Text(placeholder='Course Name', description='Course Name:', disabled=False)

  DELC_desc_label = Label("Course Description: ")
  DELC_desc_textbox = Text(description='Course Description:', disabled=False)

  DELC_quota_label = Label("Quota: ")
  DELC_quota_intbox = IntText(description='Quota:', disabled=False)

  DELC_cdate_label = Label("Course Date:")
  DELC_cdate_datepicker = DatePicker(description='Course Date:', disabled=False)

  DELC_edate_label = Label("Enrollment Deadline:")
  DELC_edate_datepicker = DatePicker(description='Enrollment Deadline:', disabled=False)

  DELC_status_label = Label("Status:") # Added label for Status
  DELC_status_dropdown = widgets.Dropdown( # Changed to Dropdown
        options=['Active', 'Not Active'], # Assuming these are the status options
        description="Status:",
        disabled=False # Enabled for display
    )

  DELC_agemin_label = Label("Age Min:")
  DELC_agemin_intbox = IntText(description='Age Min:', disabled=False)
  DELC_agemax_label = Label("Age Max:")
  DELC_agemax_intbox = IntText(description='Age Max:', disabled=False)

  DELC_remarks_label = Label("Remarks:")
  DELC_remarks_textbox = Text(description='Remarks:', disabled=False)

  DELC_category_label = Label("Category:")
  DELC_category_textbox = Text(description='Category:', disabled=False)

  DELC_tea_message_label = Label("")


# create 4 buttons
  DELC_retrieve_button = widgets.Button(description="Retrieve")
  DELC_delete_button = widgets.Button(description="Delete")
  DELC_clear_button = widgets.Button(description="Clear")
  DELC_back_button = widgets.Button(description="Back")

# define the event handlers for the three buttons when they are being clicked
  def on_DELC_retrieve_button_clicked(b):
    c.execute("SELECT * FROM COURSES WHERE cou_id = '" + DELC_id_textbox.value + "'")
    for row in c:
      DELC_name_textbox.value = row[1]
      DELC_desc_textbox.value = row[2]
      DELC_quota_intbox.value = row[3]
      DELC_cdate_datepicker.value = row[4]
      DELC_edate_datepicker.value = row[5]
      DELC_status_dropdown.value = row[6] # Assuming status is at index 6 and setting dropdown value
      DELC_agemax_intbox.value = row[7] # Assuming age min is at index 7
      DELC_agemin_intbox.value = row[8] # Assuming age max is at index 8
      DELC_remarks_textbox.value = row[9] # Assuming remarks is at index 9
      DELC_category_textbox.value = row[10] # Assuming category is at index 10


  DELC_retrieve_button.on_click(on_DELC_retrieve_button_clicked)

  def on_DELC_clear_button_clicked(b):
    DELC_id_textbox.value = ""
    DELC_name_textbox.value = ""
    DELC_desc_textbox.value = ""
    DELC_quota_intbox.value = 0
    DELC_cdate_datepicker.value = None
    DELC_edate_datepicker.value = None
    DELC_status_dropdown.value = 'Active' # Reset dropdown to default
    DELC_agemin_intbox.value = 0 # Changed to 0 for IntText
    DELC_agemax_intbox.value = 0 # Changed to 0 for IntText
    DELC_remarks_textbox.value = "" # Clear Remarks
    DELC_category_textbox.value = "" # Clear Category
    DELC_tea_message_label.value = ""

  DELC_clear_button.on_click(on_DELC_clear_button_clicked)

  def on_DELC_delete_button_clicked(b):
    c.execute(
    """
          DELETE FROM COURSES
          WHERE cou_id = '{}'
    """.format(DELC_id_textbox.value)
    )
    c.execute("commit")
    DELC_tea_message_label.value = "Delete Success!"
  DELC_delete_button.on_click(on_DELC_delete_button_clicked)

  def on_DELC_back_button_clicked(b):
    showPage("Manager Dashboard")
  DELC_back_button.on_click(on_DELC_back_button_clicked)

# display the objects
  DELC_line1HBox = widgets.HBox([DELC_id_textbox, DELC_retrieve_button])
  DELC_line2HBox = widgets.HBox([DELC_spacer])
  DELC_textboxVBox = widgets.VBox([DELC_name_textbox, DELC_desc_textbox, DELC_quota_intbox, DELC_cdate_datepicker, DELC_edate_datepicker, DELC_status_dropdown, DELC_agemax_intbox, DELC_agemin_intbox, DELC_remarks_textbox, DELC_category_textbox]) # Swapped order of agemax and agemin
  DELC_line3HBox = widgets.HBox([DELC_textboxVBox])
  DELC_line4HBox = widgets.HBox([DELC_delete_button, DELC_clear_button])
  DELC_line5HBox = widgets.HBox([DELC_back_button])

  display(widgets.VBox([DELC_line1HBox, DELC_line2HBox, DELC_line3HBox, DELC_tea_message_label, DELC_line4HBox, DELC_line5HBox]))
################ Delete Course Tab - End ################